## Introduction to Seurat:
In the project, you will predominantly use Seurat, an R package that combines many functionalities for the
analysis of single-cell data. For the initial steps, you can find helpful documentation here. They will be explicitly
mentioned if other packages are required for specific tasks.

Before you start programming, you should set up the system as follows:

## System-setup:
1. Install Conda

2. Install all packages using the provided environment.yml file. [Mac users: delete singleR line from the
.yml file before running the following command]
conda env create -f environment.yml

3. Start the conda environment with:
conda activate single-cell

4. Install CellChat by starting R and install CellChat using devtools
devtools::install_github("sqjinCellChat")

If you are using a Mac, delete the line for singleR from environment.yml file and manually install
SingleR using the following commands in R:
```if (!require("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("SingleR")```

5. If you have problems with Seurat’s clustering functions, please try to downgrade Matrix and spatstat.core
packages.

`remove.packages(grep("spatstat", installed.packages(), value = T))`

`devtools::install_version("spatstat", version = "2.4.4")`

install.packages("Matrix", ".", type = "source", repos = "http://R-Forge.R-p6. Test if you have installed all necessary libraries:

```suppressPackageStartupMessages({
library(dplyr)
library(spatstat.core)
library(Seurat)
library(patchwork)
library(DoubletFinder)
library(SingleR)
library(enrichR)
library(CellChat)
library(SingleCellExperiment)
library(SeuratWrappers)
library(tidyverse)
library(monocle3)
library(celldex)
})

## Download the data:
You can download the dataset for this project under the following link: [https://icbb-share.s3.eucentral-1.amazonaws.com/single-cell-bioinformatics/scbi_ds1.zip](https://icbb-share.s3.eu-central-1.amazonaws.com/single-cell-bioinformatics/scbi_ds1.zip) The file contains the data of four samples:
BMMC_D1T1, BMMC_D1T2, CD34_D2T1 and CD34_D3T1, with separate expression matrices for each sample.